# Импорты библиотек

In [20]:
import numpy
from plotly import express as px
import time

# Метод биортогонализации

Данный метод заключается в построении двух систем линейно независимых векторов $p_0, p_1, \ldots, p_{n - 1}$ и $q_0, q_1, \ldots, q_{n - 1}$ таких, что $(Ap_i, q_j) = 0, i \neq j$ и $(Ap_i, q_i) \neq 0.\\$
Вектора $p_0$ и $q_0$ выбираются произвольно, но так, чтобы скалярное произведение было отлично от 0, остальные вектора строятся по рекуррентным соотношениям:</br>
$\displaystyle
p_1 = Ap_0 - \gamma_0 p_0 \,\,\,\,\,\,\,\,\,\, q_1 = A^T q_0 - \gamma_0 q_0 \\
p_{i + 1} = Ap_i - \gamma_i p_i - \delta_i p_{i - 1} \,\,\,\,\,\,\,\,\,\, q_{i + 1} = A^T q_i - \gamma_i q_i - \delta_i q_{i - 1}, \\$
где </br>
$\displaystyle
\gamma_i = \frac{(Ap_i, A^T q_i)}{(Ap_i, q_i)} \\
\delta_i = \frac{(Ap_i, q_i)}{(Ap_{i - 1}, q_{i - 1})} \\
$
В случае, если $(p_r, q_r) = 0, r < n$, то, если $p_r = q_r = 0$, тогда можно достроить систему. Определяются новые векторы $p_0^{(1)}, q_0^{(1)}$, которые строятся по формулам:</br>
$\displaystyle
p_0^{(1)} = y - \sum_{i = 0}^{r - 1} \frac{(q_i, Ay)}{(Ap_i, q_i)} p_i \\
q_0^{(1)} = y - \sum_{i = 0}^{r - 1} \frac{(y, Ap_i)}{(Ap_i, q_i)} q_i, \\
$где $y$ $-$ произвольный вектор.</br>
После аналогично строим остальные вектора, если вектора снова получаются нулевые, то повторяется процесс, пока не будет получено $2n$ векторов. В случае если $(p_r, q_r) = 0$ и при этом хотя бы один из них ненулевой, то выбираются новые $p_0$ и $q_0$, далее система строится заново.</br>
После построения системы ответ на решение системы $Ax = b$ получается так:</br>
$\displaystyle
x = \sum_{i = 0}^{n - 1} \frac{(b, q_i)}{(Ap_i, q_i)} p_i
$

In [21]:
def gamma_i(i: int, A: numpy.array, P: numpy.array, Q: numpy.array) -> float | None:
    return numpy.dot(A @ P[i], A.T @ Q[i]) / numpy.dot(A @ P[i], Q[i])

def delta_i(i: int, A: numpy.array, P: numpy.array, Q: numpy.array) -> float:
    return numpy.dot(A @ P[i], Q[i]) / numpy.dot(A @ P[i - 1], Q[i - 1])

def get_new_p_0_and_q_0(A: numpy.array, P: numpy.array, Q: numpy.array, i: int) -> tuple[numpy.array, numpy.array]:
    while True:
        y = numpy.random.rand(A.shape[0])

        p_0 = y - sum(numpy.dot(Q[j], A @ y) / numpy.dot(A @ P[j], Q[j]) * P[j] for j in range(i))
        q_0 = y - sum(numpy.dot(y, A @ P[j]) / numpy.dot(A @ P[j], Q[j]) * Q[j] for j in range(i))

        if numpy.dot(p_0, q_0) != 0:
            break

    return p_0, q_0

def build_bases(A: numpy.array) -> tuple[numpy.array, numpy.array]:
    n = A.shape[0]
    P = numpy.zeros((n, n))
    Q = numpy.zeros((n, n))

    P[0] = Q[0] = numpy.random.rand(n)

    if numpy.dot(P[0], Q[0]) == 0:
        return None
    
    if n == 1:
        return P, Q

    P[1] = A @ P[0] - gamma_i(0, A, P, Q) * P[0]
    Q[1] = A.T @ Q[0] - gamma_i(0, A, P, Q) * Q[0]

    if numpy.dot(P[1], Q[1]) == 0:
        return None

    for i in range(2, n):
        P[i] = A @ P[i - 1] - gamma_i(i - 1, A, P, Q) * P[i - 1] - delta_i(i - 1, A, P, Q) * P[i - 2]
        Q[i] = A.T @ Q[i - 1] - gamma_i(i - 1, A, P, Q) * Q[i - 1] - delta_i(i - 1, A, P, Q) * Q[i - 2]

        if numpy.dot(P[i], Q[i]) == 0:
            if numpy.linalg.norm(P[i]) == 0 and numpy.linalg.norm(Q[i]) == 0:
                P[i], Q[i] = get_new_p_0_and_q_0(A, P, Q, i)
            else:
                return None

    return P, Q


def biorthogonalization_method(A: numpy.array, b: numpy.array) -> numpy.array:
    while True:
        res = build_bases(A)
        if res is not None:
            P, Q = res
            break
    
    x = numpy.zeros(A.shape[0])
    for i in range(A.shape[0]):
        x += numpy.dot(b, Q[i]) / numpy.dot(A @ P[i], Q[i]) * P[i]
    
    return x

# Метод Якоби

Метод заключается в том, что мы выражаем все $x_i$ из каждой строки системы $Ax = b$, соответствующей его индексу. Берем начальное приближение, обычно нулевой вектор для всех неизвестных. Затем все неизвестные вычисляются с помощью выраженных формул, в которые подставляем значения с прошлой итерации (на первом этапе — 0 для всех неизвестных). Постепенно все неизвестные сходятся к своим настоящим корням.</br>
Для системы $Ax = b$, где
$A = (a_{ij})_{n \times n},
x = (x_1, x_2, \dots, x_n)^T,
b = (b_1, b_2, \dots, b_n)^T,$ каждая переменная выражается так:

$\displaystyle
x_i = \frac{1}{a_{ii}} \left( b_i - \sum_{\substack{j=1\\ j \neq i}}^{n} a_{ij} x_j \right), \quad i = 1, 2, \dots, n
$, где $n$ - размерность матрицы

In [22]:
def jacobi_method(A: numpy.array, b: numpy.array, eps: float = 1e-4, max_iter: int = 100) -> numpy.array:
    """
    A: матрица коэффициентов (n x n)\n
    b: вектор правых частей (n)\n
    eps: точность\n
    max_iter: максимальное число итераций\n
    return: приближенное решение x
    """

    n = A.shape[0]

    # Проверка ненулевых диагональных элементов
    if numpy.any(numpy.diag(A) == 0):
        raise ValueError("На диагонали матрицы присутствует нулевой элемент")

    # Начальное приближение
    x = numpy.zeros(n)
    x_new = numpy.zeros(n)

    for _ in range(max_iter):
        for i in range(n):
            s = 0.0
            for j in range(n):
                if j != i:
                    s += A[i, j] * x[j]

            x_new[i] = (b[i] - s) / A[i, i]

        # Проверка сходимости
        if numpy.linalg.norm(x_new - x) < eps:
            return x_new

        x[:] = x_new

    return x

# Анализ методов

## 1. Скорость выполнения

In [23]:
def runtime_check(A: numpy.array, b: numpy.array, method: callable) -> tuple[float, numpy.array]:
    start = time.time()
    x = method(A.copy(), b.copy())
    end = time.time() - start

    return end, x

## 2. Погрешность решения

In [24]:
def error_check(x: numpy.array, x_true: numpy.array) -> float:
    return numpy.linalg.norm(x - x_true)

## 3. Устойчивость решения

In [25]:
def stability_check(A: numpy.array, b: numpy.array, x: numpy.array, n: int, method: callable, epsilon: float) -> float:
    delta_b = numpy.random.rand(n)
    delta_b = delta_b / numpy.linalg.norm(delta_b) * epsilon

    delta_x = method(A.copy(), b + delta_b)

    solution_relative_perturbation = numpy.linalg.norm(delta_x - x) / numpy.linalg.norm(x)
    return solution_relative_perturbation

## Построение графика

In [26]:
def show_graphic(sizes: list[int], by_biorthogonalization: list[float], by_jacobi: list[float], title: str, xaxis_title: str, yaxis_title: str) -> None:
    fig = px.line()

    fig.add_scatter(x=sizes, y=by_biorthogonalization, name="метод биортогонализация")
    fig.add_scatter(x=sizes, y=by_jacobi, name="метод Якоби")

    fig.update_layout(
        title = title,
        xaxis_title = xaxis_title,
        yaxis_title = yaxis_title
    )

    fig.show()

## Вывод

In [27]:
def analysis(max_n: int, epsilon: float) -> None:
    sizes = [i for i in range(1, max_n + 1)]

    times_biorthogonalization = []
    times_jacobi = []

    errors_biorthogonalization = []
    errors_jacobi = []

    stability_biorthogonalization = []
    stability_jacobi = []

    for n in sizes:
        # Генерация случайной системы Ax = b
        def generate_diagonally_dominant_matrix(n: int) -> numpy.ndarray:
            A = numpy.random.rand(n, n)
            for i in range(n):
                A[i, i] += numpy.sum(numpy.abs(A[i]))
            return A

        A = generate_diagonally_dominant_matrix(n)
        x_true = numpy.random.rand(n)
        b = A @ x_true
        
        # Биортогонализация
        t, x = runtime_check(A, b, biorthogonalization_method)
        times_biorthogonalization.append(t)
        errors_biorthogonalization.append(error_check(x, x_true))
        stability_biorthogonalization.append(stability_check(A, b, x, n, biorthogonalization_method, epsilon))

        # Якоби
        t, x = runtime_check(A, b, jacobi_method)
        times_jacobi.append(t)
        errors_jacobi.append(error_check(x, x_true))
        stability_jacobi.append(stability_check(A, b, x, n, jacobi_method, epsilon))

    show_graphic(
        sizes=sizes, 
        by_biorthogonalization=times_biorthogonalization,
        by_jacobi=times_jacobi,
        title="График скорости вычисления", 
        xaxis_title="Размер матрицы n",
        yaxis_title="Время выполнения (сек)"
    )
    show_graphic(
        sizes=sizes, 
        by_biorthogonalization=errors_biorthogonalization,
        by_jacobi=errors_jacobi,
        title="График точности методов", 
        xaxis_title="Размер матрицы n", 
        yaxis_title="Погрешность ||x - x_true||"
    )
    show_graphic(
        sizes=sizes, 
        by_biorthogonalization=stability_biorthogonalization,
        by_jacobi=stability_jacobi,
        title=f"График устойчивости решения при возмущении вектора b на {epsilon * 100}%",
        xaxis_title="Размер матрицы n",
        yaxis_title="Относительное возмущение решения (в процентах)"
    )

In [28]:
analysis(500, 1e-3)

/tmp/ipykernel_218835/3197737393.py:2: RuntimeWarning:

overflow encountered in dot

/tmp/ipykernel_218835/3197737393.py:2: RuntimeWarning:

invalid value encountered in dot

/tmp/ipykernel_218835/3197737393.py:42: RuntimeWarning:

invalid value encountered in dot

/tmp/ipykernel_218835/3197737393.py:39: RuntimeWarning:

invalid value encountered in matmul

/tmp/ipykernel_218835/3197737393.py:2: RuntimeWarning:

invalid value encountered in matmul

/tmp/ipykernel_218835/3197737393.py:5: RuntimeWarning:

invalid value encountered in matmul

/tmp/ipykernel_218835/3197737393.py:40: RuntimeWarning:

invalid value encountered in matmul

/tmp/ipykernel_218835/3197737393.py:60: RuntimeWarning:

invalid value encountered in dot

/tmp/ipykernel_218835/3197737393.py:60: RuntimeWarning:

invalid value encountered in matmul

/tmp/ipykernel_218835/3197737393.py:5: RuntimeWarning:

overflow encountered in dot

/tmp/ipykernel_218835/3197737393.py:5: RuntimeWarning:

invalid value encountered in dot

